In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [172]:
df=pd.read_csv('/content/data_cleaned.csv')
#print(df.info())
#print(df.head())
print(df.columns)

Index(['Name', 'Ticker', 'Sub-Sector', 'Market Cap_x', 'PE Ratio_x',
       '1M Return', '1W Return', '6M Return', '1Y Return', '5Y CAGR',
       'PE Ratio_y', 'PB Ratio', 'PS Ratio', 'EV/EBITDA Ratio',
       'EV / Revenue Ratio', 'Price / Sales', 'Price / Free Cash Flow',
       'Market Cap_y', 'Enterprise Value', 'Dividend Yield',
       'Dividend Per Share', 'Earnings Per Share', 'Payout Ratio',
       'Free Cash Flow', 'Net Profit Margin', 'EBITDA Margin',
       'Return on Assets', 'Return on Equity', 'Return on Investment', 'ROCE',
       'Cash Flow Margin', 'Operating Cash Flow', 'Inventory Turnover Ratio',
       'Asset Turnover Ratio', 'Working Capital Turnover Ratio',
       'Days of Sales Outstanding', 'Debt to Equity',
       'Long Term Debt to Equity', 'Interest Coverage Ratio',
       '3Y Historical Dividend Growth', '5Y Historical Revenue Growth',
       '5Y Historical EBITDA Growth', '5Y Historical EPS Growth',
       '1Y Historical Revenue Growth', '1Y Historical EBIT

**Constant Constraints**


In [161]:
df['Market Cap'] = df['Market Cap_y']
df_copy = df[df['Market Cap'] > 5000]

df_copy=df_copy[
    (df['Free Cash Flow'] > 0) &
    (df['Return on Equity'] >= 7) &
    (df['Net Profit Margin'] >= 7) &
    (df['Interest Coverage Ratio'] > 2)
]
#print(df_copy.info())

**Sector-Wise Constraints**

In [162]:
df['Market Cap'] = df['Market Cap_y']
df = df[df['Market Cap'] > 5000]

sector_median_npm = df.groupby('Sub-Sector')['Net Profit Margin'].transform('median')

filtered_df = df[
    (df['Free Cash Flow'] > 0) &
    (df['Return on Equity'] >= 7) &
    (df['Net Profit Margin'] > sector_median_npm) &
    (df['Interest Coverage Ratio'] > 2)
]
#print(len(filtered_df))
#print(filtered_df.info())
pd.set_option('display.max_row', None)
#print(filtered_df["Name"])

In [163]:
filtered_df["Score"] = (
    filtered_df["Return on Equity"].rank(ascending=False) +
    filtered_df["Net Profit Margin"].rank(ascending=False) +
    filtered_df["Interest Coverage Ratio"].rank(ascending=False) +
    filtered_df["Free Cash Flow"].rank(ascending=False)
)
filtered_df = filtered_df.sort_values("Score")
#print(filtered_df["Name"])'
print(filtered_df.columns)



Index(['Name', 'Ticker', 'Sub-Sector', 'Market Cap_x', 'PE Ratio_x',
       '1M Return', '1W Return', '6M Return', '1Y Return', '5Y CAGR',
       'PE Ratio_y', 'PB Ratio', 'PS Ratio', 'EV/EBITDA Ratio',
       'EV / Revenue Ratio', 'Price / Sales', 'Price / Free Cash Flow',
       'Market Cap_y', 'Enterprise Value', 'Dividend Yield',
       'Dividend Per Share', 'Earnings Per Share', 'Payout Ratio',
       'Free Cash Flow', 'Net Profit Margin', 'EBITDA Margin',
       'Return on Assets', 'Return on Equity', 'Return on Investment', 'ROCE',
       'Cash Flow Margin', 'Operating Cash Flow', 'Inventory Turnover Ratio',
       'Asset Turnover Ratio', 'Working Capital Turnover Ratio',
       'Days of Sales Outstanding', 'Debt to Equity',
       'Long Term Debt to Equity', 'Interest Coverage Ratio',
       '3Y Historical Dividend Growth', '5Y Historical Revenue Growth',
       '5Y Historical EBITDA Growth', '5Y Historical EPS Growth',
       '1Y Historical Revenue Growth', '1Y Historical EBIT

In [164]:
import numpy as np

def buffett_score(row):
    score = 0

    # ROE > 15%: 20 points
    if row.get('Return on Equity', 0) > 15:
        score += 20

    # Debt to Equity < 0.5: 15 points
    if row.get('Debt to Equity', np.inf) < 0.5:
        score += 15

    # PE Ratio < 25: 15 points
    if row.get('PE Ratio_x', np.inf) < 25:
        score += 15

    # 5Y EPS Growth > 10%: 10 points
    if row.get('5Y Historical EPS Growth', 0) > 10:
        score += 10

    # Free Cash Flow > 0: 10 points
    if row.get('Free Cash Flow', -1) > 0:
        score += 10

    # Net Profit Margin > 10%: 10 points
    if row.get('Net Profit Margin', 0) > 10:
        score += 10

    # ROCE > 15%: 10 points
    if row.get('ROCE', 0) > 15:
        score += 10

    # Dividend Yield > 1% (optional): 5 points
    if row.get('Dividend Yield', 0) > 1:
        score += 5

    return score


In [165]:
filtered_df['Buffett Score'] = filtered_df.apply(buffett_score, axis=1)
df_sorted = filtered_df.sort_values(by='Buffett Score', ascending=False)
print(df_sorted[['Name', 'Ticker', 'Buffett Score']].head(20))


                                   Name      Ticker  Buffett Score
10                                  itc         ITC             95
76                                cipla       CIPLA             95
408                                nava        NAVA             95
6                               infosys        INFY             95
444      great eastern shipping company      GESHIP             95
563                maharashtra seamless  MAHSEAMLES             95
99                   zydus lifesciences   ZYDUSLIFE             95
712                                ghcl        GHCL             95
742             sharda motor industries  SHARDAMOTR             95
613                    cms info systems     CMSINFO             95
588              banco products (india)  BANCOINDIA             95
340                    karur vysya bank  KARURVYSYA             95
349               kansai nerolac paints   KANSAINER             95
79              dr reddy's laboratories     DRREDDY           

In [166]:
# Avoid divide by zero
filtered_df['Earnings Yield'] = 1 / filtered_df['EV/EBITDA Ratio'].replace(0, float('nan'))
filtered_df['ROIC'] = filtered_df['ROCE']  # Using ROCE as proxy

filtered_df['Greenblatt Rank'] = (
    filtered_df['Earnings Yield'].rank(ascending=False, method='min') +
    filtered_df['ROIC'].rank(ascending=False, method='min')
)

filtered_df['Greenblatt Score'] = 100 - filtered_df['Greenblatt Rank'].rank(method='min') * (100 / len(filtered_df))


In [167]:
def piotroski_score(row):
    score = 0
    if row.get('Net Profit Margin', -1) > 0: score += 1
    if row.get('Return on Assets', -1) > row.get('5Y Avg Return on Assets', -2): score += 1
    if row.get('Operating Cash Flow', -1) > 0: score += 1
    if row.get('Debt to Equity', 1e6) < row.get('Long Term Debt to Equity', 1e6): score += 1
    if row.get('Total Assets', 0) > 0 and row.get('Total Liabilities', 0) > 0:
        if (row['Total Assets'] / row['Total Liabilities']) > 1.5: score += 1
    if row.get('Asset Turnover Ratio', 0) > row.get('5Y Avg Return on Assets', -2): score += 1
    return score

filtered_df['Piotroski Score'] = filtered_df.apply(piotroski_score, axis=1)


In [168]:
filtered_df['PEG Ratio'] = filtered_df['PE Ratio_x'] / filtered_df['5Y Historical EPS Growth'].replace(0, float('nan'))
filtered_df['Peter Lynch Score'] = filtered_df['PEG Ratio'].apply(
    lambda x: 100 if x < 1 else 50 if x < 2 else 0
)


In [169]:
filtered_df['Composite Score'] = (
    filtered_df['Buffett Score'] * 0.3 +
    filtered_df['Greenblatt Score'] * 0.3 +
    filtered_df['Piotroski Score'] * (100 / 6) * 0.2 +
    filtered_df['Peter Lynch Score'] * 0.2
)

filtered_df['Final Rank'] = filtered_df['Composite Score'].rank(ascending=False)


In [170]:
top_stocks = filtered_df.sort_values(by='Final Rank')
#print(top_stocks[['Name', 'Final Rank', 'Buffett Score', 'Greenblatt Score', 'Piotroski Score', 'Peter Lynch Score', 'Composite Score']])

print(len(top_stocks))

236


In [171]:
# Convert company names to lowercase for safe matching
filtered_df['Name_lower'] = filtered_df['Name'].str.lower()

# Define keywords to search
companies = ['hdfc bank', 'tata consultancy services', 'reliance industries']

# Filter rows that contain any of the keywords
filtered_df_matches = filtered_df[filtered_df['Name_lower'].str.contains('|'.join(companies), na=False)]

# Show name and rank
print(filtered_df_matches[['Name', 'Final Rank', 'Composite Score', 'Buffett Score', 'Greenblatt Score', 'Piotroski Score']].sort_values(by='Final Rank'))


                        Name  Final Rank  Composite Score  Buffett Score  \
2  tata consultancy services        96.0        62.807910             70   
1                  hdfc bank       136.0        54.553672             85   
0        reliance industries       217.0        31.079096             35   

   Greenblatt Score  Piotroski Score  
2         94.915254                4  
1         19.067797                4  
0         24.152542                4  
